- 回帰分析
    - ロジスティック回帰

- 分析流れ
    - 準備
    - モデル構築・学習
    - 評価

In [6]:
import requests, io

# read dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
res = requests.get(url).content
adult = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)
adult.columns =['age','workclass','fnlwgt','education','education-num','marital-status',
                             'occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                             'native-country','flg-50K']
adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,flg-50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


収入が50Kを超えるかどうかを示す目的変数はflg-50Kです。データの値は「<=50K」と「>50K」で、このままでは扱いにくいので、0/1のフラグ変数に変換します。

In [7]:
# transform data
adult['fin_flg'] = adult['flg-50K'].map(lambda x: 1 if x ==' >50K' else 0)
adult.groupby('fin_flg').size()

fin_flg
0    24720
1     7841
dtype: int64

### ロジスティック回帰モデル

***準備***

In [19]:
# split train data and test data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# setting X and Y
X = adult[['age','fnlwgt','education-num','capital-gain','capital-loss']]
Y = adult['fin_flg']

#split train data and test data to 8:2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 0)

# standard
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


***学習***

In [20]:
# read model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [21]:
#learning
 
model.fit(X_train, Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

学習結果

In [22]:
# coefficient
model.coef_

array([[0.53744404, 0.03466307, 0.86071867, 2.44592366, 0.28340329]])

In [23]:
#intercept
model.intercept_

array([-1.27059108])

***評価***  
評価データを用いて学習結果の当てはまりがどの程度のものなのか評価します

In [24]:
#predict
print('正解率(train):{:.3f}'.format(model.score(X_train, Y_train)))
print('正解率(test):{:.3f}'.format(model.score(X_test, Y_test)))

正解率(train):0.811
正解率(test):0.806


上記の結果から、訓練データとテストデータともに約80%の正解率であり、過学習は起きていないと判断できます。